# Optimizinzg model parameters

## Prerequisite Code
[02_Dataset_and_DataLoader](02_Dataset_and_DataLoader.ipynb) 과 [04_BuildTheNeuralNetwork.ipynb](04_BuildTheNeuralNetwork.ipynb)에서 코드를 가져왔다

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

device = "cpu"

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

## Hyperparameter
- epoch: the number times of iterate of the optimizaion loop
- batch size: the number of data samples propagated through the nework befoe parameters are updated
- learning rate: how much to update models parameters at each batch/epoch.
  - Smaller values: slow learning speed
  - Large values: may result in unpredictable behavior

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimizaion Loop
Each iteration of the optimizaion loop is called an epoch.   
Each epoch consists of two main parts
- The Train Loop: iterate over the training dataset and try to converge to opimal parameters.
- The Validation/Test Loop: iterate over the test dataset to check if model performance is improving


### Loss Function
- Loss Function: measures the degree of dissimilarity of obtained result to the target value
- Common loss function
  - ```nn.MSELoss```(Mean Square Error) for regression
  - ```nn.NLLLoss```(Negative Log Likelihood) for classification
  - ```nn.CrossEntropyLoss```: combines ```nn.NLLLoss``` and ```nn.LogSoftmax```

In [3]:
loss_fn = nn.CrossEntropyLoss()

### Optimizer
Optimization is the process of adjusting model parameters to reduce model error in each training step.
- Optimization algorithms: Define how this process is performed.
  - ```torch.optim.SGD```(Stochastic Gradient Descent)

In [4]:
optimizer =torch.optim.SGD(model.parameters(), lr=learning_rate)


### Optimization Step
- Call ```optimizer.zero_grad()``` to reset the gradients of model parameters.
  - Gradients by default add up; to prevent doublecounting, we explicitly zero them at each iteration.
- Call ```loss.backwards()``` to Backpropagated the prediction loss.
  - Pytorch deposits the gradients of the loss w,r,t each parameter.
- Call ```optimizer.step()``` to adjust the parameters by the gradients callected in the packward pass.

## Full Implementaion
- Define ```train_loop```: loops over optimization code.
- Define ```test_loop```: Evaluates the model's performance against test data

```enumerate```: 순서가 있는 자료형(list, set, tuple, dictionary, string, etc...)를 받아 index를 포함하는 enumerate 객체(index, feature)를 return 해줌

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (x,y) in enumerate(dataloader):
        pred = model(x.to(device))
        loss = loss_fn(pred,y.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x.to(device))
            test_loss += loss_fn(pred.to(device), y.to(device)).item()
            correct += (pred.argmax(1).to("cpu") == y.to("cpu")).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    

- Initialize the loss function and optimizer
- Pass los function and optimizer to ```train_loop``` and ```test_loop```.
- Fell Free to increase the number of epochs to track the model's improving performance


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10

for t in range(epochs):
    print(f"Epch {t+1}\n----------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epch 1
----------------------------------
loss: 2.319158 [    0/60000]
loss: 2.298460 [ 6400/60000]
loss: 2.276989 [12800/60000]
loss: 2.258914 [19200/60000]
loss: 2.266488 [25600/60000]
loss: 2.220623 [32000/60000]
loss: 2.238683 [38400/60000]
loss: 2.207476 [44800/60000]
loss: 2.194676 [51200/60000]
loss: 2.158496 [57600/60000]
Test Error: 
 Accuracy: 25.9%, Avg loss: 2.159938 

Epch 2
----------------------------------
loss: 2.175169 [    0/60000]
loss: 2.163112 [ 6400/60000]
loss: 2.105735 [12800/60000]
loss: 2.111700 [19200/60000]
loss: 2.091763 [25600/60000]
loss: 2.013721 [32000/60000]
loss: 2.045590 [38400/60000]
loss: 1.970821 [44800/60000]
loss: 1.957163 [51200/60000]
loss: 1.896051 [57600/60000]
Test Error: 
 Accuracy: 60.5%, Avg loss: 1.899842 

Epch 3
----------------------------------
loss: 1.926307 [    0/60000]
loss: 1.899833 [ 6400/60000]
loss: 1.789111 [12800/60000]
loss: 1.822388 [19200/60000]
loss: 1.742253 [25600/60000]
loss: 1.673792 [32000/60000]
loss: 1.693270 [